In [1]:
import os
import requests
import os
import sys
from tqdm import tqdm
import time
import Cobol2Spark as chat
import numpy as np

In [2]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().strip()
    data = ' '.join(data.split())

In [3]:
generation,code, usage = chat.cobol2spark(data)

100%|██████████| 3/3 [03:06<00:00, 62.13s/it]


In [4]:
print(usage)

0.32281800000000005


In [5]:
print(generation)

from pyspark.sql import SparkSession

# Creating Spark Session
spark = SparkSession.builder.appName('COBOL_to_PySpark').getOrCreate()

# Assigning files to variables
ARQPARM = spark.read.text("path_to_ARQPARM_file")
ARQSORT = spark.read.text("path_to_ARQSORT_file")
DIMESTIP = spark.read.text("path_to_DIMESTIP_file")
ARQFTP = spark.read.text("path_to_ARQFTP_file")



 from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cobolToPyspark').getOrCreate()


df = spark.read.csv('data.csv', header=True, inferSchema=True)


df.createOrReplaceTempView("table")
result = spark.sql("SELECT * FROM table WHERE column_name IN (875, 876, 878)")



 from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a SparkSession
spark = SparkSession.builder.appName("COBOL_Translation").getOrCreate()

# Read the data from CSV file (assuming the data from `ATSAUDAO.SSCESTIPULANTE` is exported to a CSV file)
df = spark.read.format("csv").option("header", "true").loa

In [6]:
print(code)

The provided PySpark code seems to be equivalent to the COBOL code. It reads the data from a CSV file, applies the same filtering and sorting logic as the COBOL code, and writes the result to a new CSV file.

Here is the corrected PySpark code:

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a SparkSession
spark = SparkSession.builder.appName("COBOL_Translation").getOrCreate()

# Read the data from CSV file (assuming the data from `ATSAUDAO.SSCESTIPULANTE` is exported to a CSV file)
df = spark.read.format("csv").option("header", "true").load("data.csv")

# Implement the filtering logic
df = df.filter((col("ETP_RMO").isin([875, 876, 878])) & \
               ((col("ETP_AMD_CANCEL") == 0) | (col("ETP_AMD_CANCEL") >= col("DATA_INI_SEL"))) & \
               (col("ETP_DT_INI") <= col("DATA_FIM_SEL")))

# Implement the sorting logic
df = df.sort(["NUM_CGC_SORT", "DATA_CANCEL_SORT", "DATA_INIC_SORT", "COD_CIA_SORT", "COD_APOLICE_SORT"])

# Writ

In [ ]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [ ]:
"""  code:
  ```
  {code}
  ```

  Your task is to do the following steps:

  1. Analyze and comprehend the COBOL code snippet above.
  2. Write an equivalent PySpark code.
  3. The equivalent code should provide the exact same output as the COBOL code.
  4. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
  5. Pay extra attention in the every data processing steps like filtering, aggregation, and sorting.
  6. Return only the PySpark code."""
usage = []
responses = []
for code in tqdm(final):
    response = chat.generate(prompt=f"""Given the following code snippet. Your role will be to decipher the code, create an equivalent PySpark code with the same functionality, including all the necessary processes.

code:
```
{code}
```

Your task is to do the following steps:

1. Analyze and comprehend the COBOL code snippet above.
2. Write an equivalent PySpark code.
3. The equivalent code should provide the exact same output as the COBOL code.
4. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
5. Pay extra attention in the every data processing steps like filtering, aggregation, and sorting.
6. Return only the PySpark code.
""",temperature=0.4, max_tokens=1000)
    responses.append(response)
    usage.append(((response.usage.prompt_tokens/1000) * 0.006)+((response.usage.completion_tokens/1000) * 0.012))
print(np.sum(usage))

In [ ]:
final_sparks = '\n\n'.join(resp.choices[0].message.content for resp in responses)

In [ ]:
with open("../Outputs/Cobol2Spark_teste_35.md", "w") as text_file:
    text_file.write(final_sparks)

In [ ]:
import re

sparks = []
for resp in  responses:
    code = '\n\n'.join(re.findall(r'```python\n(.*?)```', resp.choices[0].message.content, re.DOTALL))
    sparks.append(code) 
final_sparks = '\n\n\n '.join(sparks)

In [ ]:
final_code = chat.generate(prompt=f"""Analise the following codes snippet and decipher the code.
                            

cobol code:
```
{data}
```

Pyspark code:
```
{final_sparks}
```

Your task is to:

1. Analyze and comprehend the PySpark and cobol code snippet above.
2. verify if in the Spark code there is any mistake or if it is not equivalent to the COBOL code.
3. If there is any mistake, correct it.
4. If the code is equivalent, return the PySpark code.
5. pay extra attention to the data types and the every data processing steps like filtering, aggregation, and sorting.
6. Return only the PySpark code.""",temperature=0.4,max_tokens=1000)

print(((final_code.usage.prompt_tokens/1000) * 0.006)+((final_code.usage.completion_tokens/1000) * 0.012))

In [ ]:
with open("../Outputs/Cobol2Spark_35.md", "w") as text_file:
    text_file.write(final_code.choices[0].message.content)

In [ ]:
chat.